<a href="https://colab.research.google.com/github/dipanshuhaldar/binary_classification_pytorch/blob/master/pytorch_mlp_binary_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* We will use the [**Ionosphere binary (two class) classification dataset**](https://archive.ics.uci.edu/ml/machine-learning-databases/ionosphere/) to demonstrate an MLP for binary classification.
* This dataset involves predicting whether there is a structure in the atmosphere or not given radar returns.

In [1]:
'''
set working directory
look in the files in the working directory
'''
#mounting google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#requirements
from numpy import vstack
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score 

In [0]:
#requirements(PyTorch)
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import BCELoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_

**Loading and Data Preparation**

In [0]:
#dataset definition
class CSVDataset(Dataset):

  #load the data set
    def __init__(self, path):
      #load the data set
      df = read_csv(path)
      #store the features and the target
      self.X = df.values[:, df.columns != 'Class']#desection on column name, here Class
      self.y = df.values[:, df.columns == 'Class']#selection based on column name, here Class
      #ensure input data is floats
      self.X = self.X.astype('float32')
      #label encode target and ensure the values are floats
      self.y = LabelEncoder().fit_transform(self.y)
      self.y = self.y.astype('float32')
      self.y = self.y.reshape((len(self.y), 1))

    #number of rows in the dataset
    def __len__(self):
      return len(self.X)

    #get a row by index
    def __getitem__(self, idx):
      return [self.X[idx], self.y[idx]]

    #get indexes for train and test rows
    def get_splits(self, n_test=0.33):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])

**Define MLP Model Framework**

In [0]:
class MLP(Module):

  #define model elements
  def __init__(self, n_inputs):
    super(MLP, self).__init__()

    #input to first hidden layer
    self.hidden1 = Linear(n_inputs, 10)
    kaiming_uniform_(self.hidden1.weight, nonlinearity = 'relu')
    self.act1 = ReLU()

    #input to second hidden layer
    self.hidden2 = Linear(10, 8)
    kaiming_uniform_(self.hidden2.weight, nonlinearity = 'relu')
    self.act2 = ReLU()

    #input to third hidden layer and output
    self.hidden3 = Linear(8, 1)
    kaiming_uniform_(self.hidden3.weight)
    self.act3 = Sigmoid()

  #forward propagate input
  def forward(self, X):
    
    #input to first hidden layer
    X = self.hidden1(X)
    X = self.act1(X)

    #second hidden layer      
    X = self.hidden2(X)
    X = self.act2(X)

    #third hidden ayer and output
    X = self.hidden3(X)
    X = self.act3(X)

    return X

**Dataset Preparation**

In [0]:
def prepare_data(path):

  #load dataset
  dataset = CSVDataset(path)

  #split calculation
  train, test = dataset.get_splits()

  #prepare data loaders
  train_dl = DataLoader(train, batch_size = 32, shuffle = True)
  test_dl = DataLoader(test, batch_size = 1024, shuffle = False)

  return train_dl, test_dl

**Train MLP Model**

In [0]:
def train_model(train_dl, model):

  #define optimization
  criterion = BCELoss()
  optimizer = SGD(model.parameters(), lr = 0.01, momentum = 0.9)

  #enumerate epochs
  epochs = 100
  for epoch in range(epochs):
    #enumerate mini batches
    for idx, (inputs, targets) in enumerate(train_dl):
      #clear the gradients
      optimizer.zero_grad()
      #compute model output
      yhat = model(inputs)
      #calculate loss
      loss = criterion(yhat, targets)
      #back propagation
      loss.backward()
      #update model weights
      optimizer.step() 

**Model Evaluation**

In [0]:
def evaluate_model(test_dl, model):

  predictions, actuals = list(), list()
  for idx, (inputs, targets) in enumerate(test_dl):
    #evaluate model on test set
    yhat = model(inputs)
    #retrieve numpy array
    yhat = yhat.detach().numpy()
    actual = targets.numpy()
    actual = actual.reshape((len(actual), 1))
    #round to class values
    yhat = yhat.round()
    #store the predicted and actuals in the empty lists
    predictions.append(yhat)
    actuals.append(actual)

  predictions, actuals  = vstack(predictions), vstack(actuals)
  #calculate accuracy
  acc = accuracy_score(actuals, predictions)
  return acc

**Make Prediction for Single Data Instance**

In [0]:
def predict(row, model):

  #convert row to Tensor
  row = Tensor([row])
  #make prediction
  yhat = model(row)
  #retrieve numpy array
  yhat = yhat.detach().numpy()
  return yhat.round()

**Code Run**

In [39]:
import os
path = '/content/drive/My Drive/ionosphere/'
os.chdir(path)
os.listdir(path)

['Ionosphere.csv',
 'original_code.ipynb',
 'pytorch_mlp_binary_classification.ipynb']

In [40]:
data_path = path + 'Ionosphere.csv'
#prepare data
train_dl, test_dl = prepare_data(data_path)
print(f'No. of rows in Train Set: {len(train_dl.dataset)} \n No. of rows in Test Set: {len(test_dl.dataset)}')

No. of rows in Train Set: 235 
 No. of rows in Test Set: 116


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
#define Neural Network
model = MLP(34)

In [0]:
#train the model
train_model(train_dl, model)

In [43]:
#model evaluation
acc = evaluate_model(test_dl, model)
print('Accuracy: %.3f' % acc)

Accuracy: 0.905


In [44]:
#make prediction for single data instance
row = [1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1,0.03760,0.85243,-0.17755,0.59755,-0.44945,0.60536,-0.38223,0.84356,-0.38542,0.58212,-0.32192,0.56971,-0.29674,0.36946,-0.47357,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300]
yhat = predict(row, model)
print('Predicted: %.3f (class=%d)' % (yhat, yhat.round()))

Predicted: 1.000 (class=1)
